# [WORK IN PROGRESS]
# Temporal Attention Model for Neural Machine Translation
Unofficial implementation of paper: http://arxiv.org/abs/1608.02927

### Requirements:
 - [Keras](https://github.com/fchollet/keras)
 - [Tensorflow](https://github.com/tensorflow/tensorflow)
 - [Theano](https://github.com/Theano/Theano)
 - https://github.com/farizrahman4u/seq2seq Seq2Seq implemtation built on top of Keras

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout, RepeatVector, Permute, Activation, recurrent, LSTM, GRU
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed, Bidirectional
from data_utils import *
from dataset import *

Using TensorFlow backend.


In [4]:
data_dir = "/data/translate"
vocab_size = 20000
en, fr = prepare_date(data_dir, vocab_size)
print("en vocab size", en.vocab_size)
print("fr vocab size", fr.vocab_size)
print("en max lentgh", en.max_length)
print("fr max length", fr.max_length)

en vocab size 19905
fr vocab size 19915
en max lentgh 110
fr max length 126


### Embeding layer for en and fr 

In [8]:
embedding_size = 64
en_set = en.matrix
fr_set = fr.matrix

In [9]:
model = Sequential()
model.add(Embedding(en.max_features, embedding_size, input_length=en.max_length, mask_zero=True))
model.compile('rmsprop', 'mse')
en_embed = model.predict(en_set)
en_embed.shape

(3000, 110, 64)

In [11]:
model = Sequential()
model.add(Embedding(fr.max_features, embedding_size, input_length=fr.max_length, mask_zero=True))
model.compile('rmsprop', 'mse')
fr_embed = model.predict(fr_set)
fr_embed.shape

(3000, 126, 64)

### Building the model

In [ ]:
# Experiment  1
hidden_size = 32
model = Sequential()
model.add(GRU(hidden_size, input_shape=(en_max_length, embedding_size))) # (3000, 110, 64) -> (3000, 32)
model.add(RepeatVector(fr_max_length)) # (3000, 32) -> (3000, 126, 32)
# model.add(TimeDistributed(Dense(fr_max_length)))
model.add(Bidirectional(GRU(embedding_size, return_sequences=True), merge_mode='sum')) # (3000, 126, 32) -> (3000, 126, 64)
model.output_shape

In [ ]:
# Experiment  2
hidden_size = 32
model = Sequential()
model.add(Embedding(en_max_features, embedding_size, input_length=en_max_length, mask_zero=True))
model.add(Bidirectional(GRU(hidden_size), merge_mode='sum'))
print(model.input_shape)
model.add(RepeatVector(fr_max_length))
model.add(GRU(embedding_size))
# print(model.output_shape)
model.add(Dense(fr_max_length, activation="softmax"))
# print(model.output_shape)
model.compile('rmsprop', 'mse')
model.get_config()

In [ ]:
model.fit(en_set, fr_set)

In [ ]:
class Translator(object):
    def __init__(self, model, source, target):
        self.model = model
        self.source = source
        self.target = target
    
    def translate(self, sentence):
        ids = sentence_to_token_ids(sentence, self.source)
        print(ids)
        x = pad_sequences([ids], maxlen=en_max_length)
        print(x)
        print(model.predict(x))
        p = [np.argmax(i) for i in model.predict(x)]
        print(p)
        return token_ids_to_sentence(p, self.target)

In [ ]:
t = Translator(model, en_vocab, fr_index)
t.translate("good morning my name is")

## Other trials

### Model infered from Seq2Seq
```python
model = Sequential()
model.add(LSTM(hidden_dim, return_sequences=True, mask_zero=True))
model.add(Dropout(droupout))
model.add(LSTM(hidden_dim, )) # Encoder
model.add(Dropout(droupout))
model.add(RepeatVecotr(output_lenght))
model.add(LSTM(hidden_dim, return_sequences=True, )) # Decoder
model.add(LSTM(hidden_dim, return_sequences=True, ))
model.add(Droupout(droupout))
model.add(TimeDistributed(Dense(output_dim)))
model.compile('rmsprop', 'mse')
```

### https://github.com/fchollet/keras/issues/395

In the model I listed below with the english sentence as input and the entire french sentence as output. The RNN model will maintain state across each timestep as it predicts the output sentence, no extra work required on your behalf. You will however need to one hot encode and zero pad the output sequence (the french sentence) and have it do a softmax over all possible words for the output at each time step. The ys then are 3D, each row is a matrix of height - number of french words, and width - number of time steps.

```python
embedding_size = 50
hidden_size = 512
output_size = 20
maxlen = 60

model = Sequential()
model.add(JZS1(embedding_size, hidden_size)) # try using a GRU instead, for fun
model.add(Dense(hidden_size, hidden_size))
model.add(Activation('relu'))
model.add(RepeatVector(maxlen))
model.add(JZS1(hidden_size, hidden_size, return_sequences=True))
model.add(TimeDistributedDense(hidden_size, output_size, activation="softmax"))

model.compile(loss='mse', optimizer='adam')
```

In [ ]:
from keras.layers.wrappers import TimeDistributed

embedding_size = 64
hidden_size = 512
embedding_size = EN_REPRESENTATION_SIZE
MAX_LEN = fr_set.shape[1]
max_features = FR_BOUND

print('Build model...')
model = Sequential()
model.add(Embedding(EN_REPRESENTATION_SIZE, EMBED_HIDDEN_SIZE, input_length=en_set.shape[1], mask_zero=True))
model.add(GRU(hidden_size)) # try using a GRU instead, for fun
model.add(Dense(hidden_size))
model.add(Activation('relu'))
model.add(RepeatVector(MAX_LEN))
model.add(GRU(hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(max_features, activation="softmax")))

model.compile(loss='mse', optimizer='adam')

### Dummy seq2seq model

In [ ]:
import seq2seq
from seq2seq.models import SimpleSeq2seq

model = Sequential()
model.add(Embedding(EN_REPRESENTATION_SIZE, EMBED_HIDDEN_SIZE, input_length=en_set.shape[1]))
model = SimpleSeq2seq(
        input_dim=EN_REPRESENTATION_SIZE,
        input_length=en_max_length,
        hidden_dim=50,
        output_length=FR_REPRESENTATION_SIZE,
        output_dim=fr_max_length)

model.compile(loss='mse', optimizer='rmsprop')

In [ ]:
# model.fit(en_embed, fr_set)

In [ ]:
# RNN = GRU
# EMBED_HIDDEN_SIZE = 50

# encoder = Sequential()
# encoder.add(Embedding(en_vocab_size, EMBED_HIDDEN_SIZE, input_length=en_max_length))

# decoder = Sequential()
# decoder.add(Embedding(fr_vocab_size, EMBED_HIDDEN_SIZE, input_length=fr_max_length))

# decoder.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
# decoder.add(RepeatVector(en_max_length))

# model = Sequential()
# model.add(Merge([encoder, decoder], mode='sum'))
# model.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
# model.add(Dropout(0.3))
# model.add(Dense(fr_vocab_size, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# p = model.predict([en_set, fr_set])

In [ ]:
# import tensorflow as tf
# outputs = [int(np.argmax(logit, axis=0)) for logit in p[0]][0:fr_input_length]
# token_ids_to_sentence(outputs, fr_index)
# # [fr_ids[output] for output in outputs]
# # print(" ".join([tf.compat.as_str(fr_ids[output]) for output in outputs]))